In [2]:
import requests
import bs4 as BeautifulSoup
import numpy as np
import pandas as pd
# import re

In [3]:
url_list=['https://ocs.ca/collections/dried-flower-cannabis',\
          'https://ocs.ca/collections/dried-flower-cannabis?page=2&hitsPerPage=12',\
          'https://ocs.ca/collections/dried-flower-cannabis?page=3&hitsPerPage=12',\
          'https://ocs.ca/collections/dried-flower-cannabis?page=4&hitsPerPage=12',\
          'https://ocs.ca/collections/dried-flower-cannabis?page=5&hitsPerPage=12',\
          'https://ocs.ca/collections/dried-flower-cannabis?page=6&hitsPerPage=12',\
          'https://ocs.ca/collections/dried-flower-cannabis?page=7&hitsPerPage=12']

In [4]:
def get_OCS_products(url_list):
    '''
    Takes a list of urls, returns a list of every product page url
    '''
    all_urls = []

    for page in url_list:
        print(page)
        source_code = requests.get(page)
        plain_text = source_code.text
        soup = BeautifulSoup.BeautifulSoup(plain_text, 'html.parser')
        
# GET STRAIN URLS FOR EACH PAGE IN THE LIST

        bowl = soup.find_all("div", {"class" : "product-tile__image"})
        for item in bowl: 
            item = item.find_next('a')
#             print(item, "\n")
            tag = item.get('href')
            url = "https://www.ocs.ca"+tag
            all_urls.append(url)

    return all_urls

In [5]:
# GET TAGS
all_urls = get_OCS_products(url_list)
all_urls

https://ocs.ca/collections/dried-flower-cannabis
https://ocs.ca/collections/dried-flower-cannabis?page=2&hitsPerPage=12
https://ocs.ca/collections/dried-flower-cannabis?page=3&hitsPerPage=12
https://ocs.ca/collections/dried-flower-cannabis?page=4&hitsPerPage=12
https://ocs.ca/collections/dried-flower-cannabis?page=5&hitsPerPage=12
https://ocs.ca/collections/dried-flower-cannabis?page=6&hitsPerPage=12
https://ocs.ca/collections/dried-flower-cannabis?page=7&hitsPerPage=12


['https://www.ocs.ca/products/balance-solei',
 'https://www.ocs.ca/products/god-bud-redecan',
 'https://www.ocs.ca/products/alien-dawg',
 'https://www.ocs.ca/products/dream-weaver',
 'https://www.ocs.ca/products/napali-cbd',
 'https://www.ocs.ca/products/purple-chitral-woodstock',
 'https://www.ocs.ca/products/banana-split',
 'https://www.ocs.ca/products/red',
 'https://www.ocs.ca/products/sativa',
 'https://www.ocs.ca/products/houndstooth',
 'https://www.ocs.ca/products/cold-creek-kush-1',
 'https://www.ocs.ca/products/super-sonic-symbl',
 'https://www.ocs.ca/products/solar-power-symbl',
 'https://www.ocs.ca/products/ultra-sour',
 'https://www.ocs.ca/products/tangerine-dream-san-rafael-71',
 'https://www.ocs.ca/products/gems',
 'https://www.ocs.ca/products/white-widow',
 'https://www.ocs.ca/products/harmonic-alta-vie',
 'https://www.ocs.ca/products/airplane-mode',
 'https://www.ocs.ca/products/50',
 'https://www.ocs.ca/products/campfire',
 'https://www.ocs.ca/products/gather',
 'https

In [9]:
def OCS_scraper(all_urls,df):
    '''
    Scrapes OCS product page for every url in the list all_urls
    '''    
    for url in all_urls:
        print(url)
        source_code = requests.get(url)
        plain_text = source_code.text
        soup = BeautifulSoup.BeautifulSoup(plain_text, 'html.parser')

    # GET strain (Name)
        strain = ""
        try:
            strain = soup.find("h1", {"class" : "product__title h2"}).text
#             print(f"Strain: {strain}")
        except:
            print(f"COULD NOT GET STRAIN NAME FOR {strain}")
            strain = "ERROR"
            
    # GET price
        price = ""
        try:
            price = soup.find("h2", {"class" : "product__price"}).text
            price = price[price.find("$")+1:price.find("/")-1]
            print(f"Price (per gram): {price}")
        except:
            print(f"COULD NOT GET PRICE FOR {strain}")
            price = "ERROR"
            
    # GET THC
        THC = ""
        try:
            bowl = soup.find("h3", {"id" : "thc-tooltip-1"})
            THC_range = bowl.find_next("p").text
            THC_range = THC_range.replace("%","")
            THC_range = THC_range.replace("-","")
            THC1, THC2 = THC_range.split()
#             print(THC1,THC2)
            THC_avg = (float(THC1)+float(THC2))/2 # GET THE AVERAGE THC
            print(THC_avg)
        except:
            print(f"COULD NOT GET AVERAGE THC FOR {strain}")
            THC = "ERROR"
            
    # GET type
        plant_type = ""
        try:
            bowl = soup.find("h3", {"id" : "plant_type-tooltip-1"})
            plant_type = bowl.find_next("p").text
            print(plant_type)
        except:
            print(f"COULD NOT GET PLANT TYPE FOR {strain}")
            plant_type = "ERROR"
            
    # GET terpenes
        terpenes = ""
        try:
            bowl = soup.find("h4", {"id" : "terpenes-tooltip-2"})
            terpenes = bowl.find_next("td").text
            terpenes = terpenes.replace(" ","")
            terpenes = terpenes.replace("\n","")
            print(terpenes)
        except:
            print(f"COULD NOT GET TERPENES FOR {strain}")
            terpenes = "ERROR"
        
    # GET description
        desription = ""
        try:
            bowl = soup.find("h3", {"class" : "product__title"})
            description = bowl.find_next("div").text
#             terpenes = terpenes.replace(" ","")
#             terpenes = terpenes.replace("\n","")
            print(description)
        except:
            print(f"COULD NOT GET DESCRIPTION FOR {strain}")
            description = "ERROR"
            
        
    # CREATE DF, APPEND DATA
        data = pd.DataFrame([[strain, url, description, price, plant_type, terpenes]], columns = df.columns)
    
        df = df.append(data, ignore_index=True)
        
    return df

Now create the dataframe that will contain the scraped information and scrape / populate the df.

In [10]:
#### SCRAPE OCS FOR EACH ITEM
OCS_df = pd.DataFrame(columns=['Strain', 'Url', 'Description','Price', 'Strain_type', 'Terpenes'])

OCS_df = OCS_scraper(all_urls,OCS_df)
OCS_df

https://www.ocs.ca/products/balance-solei
Price (per gram): 9.06
6.0
Indica dominant
Myrcene,Beta-Caryophyllene,Limonene,Beta-Pinene,Linalool
Balance from Solei is available pre-rolled and in many dried flower weights. Its dark green buds are sun-grown in a greenhouse and harvested at optimal times to ensure its signature spicy flavour, which comes from the peppery beta-caryophyllene in the strain’s terpene profile. This hybrid has very mild THC potency.

https://www.ocs.ca/products/god-bud-redecan
Price (per gram): 9.09
17.0
Hybrid
Beta-Caryophyllene,Trans-Caryophyllene,Humulene,Myrcene,Limonene
God Bud from Redecan has resin-coated buds with an herbal, musky flavour combined with tropical fruit and undertones of berry, lavender and pine. The odour can be described as tropical, earthy and citrusy (a result of the caryophyllene-rich terpene profile). This green-and-white-coloured strain with medium THC potency is greenhouse-grown in pesticide-free natural lighting conditions, and then 

Price (per gram): 13.18
9.3
Sativa dominant
Myrcene,Alpha-Pinene,Beta-Pinene,Limonene
Harmonic from AltaVie is a sativa-dominant strain with significant CBD levels for balance and mild THC potency. Its green buds are grown indoors, hand-trimmed and hang-dried indoors, and the myrcene in its terpene mix is responsible for the earthiness in its taste and aroma. AltaVie offers it as capsules (30 per bottle), as well as in 1 g, 3.5 g and 7 g dried flower varieties.

https://www.ocs.ca/products/airplane-mode
Price (per gram): 13.13
15.0
Indica dominant
Myrcene,Beta-Pinene,Terpinolene,Beta-Caryophyllene
Airplane Mode from AltaVie is hand-trimmed and hang-dried. This indoor-grown indica-dominant strain has medium THC potency and a woody and earthy aroma created by the myrcene, beta-pinene and terpinolene in its terpene mix. It’s available in 1 g, 3.5 g and 7 g dried flower varieties.

https://www.ocs.ca/products/50
Price (per gram): 11.38
15.0
Hybrid
Myrcene,Alpha-Pinene,Beta-Pinene
A classic

Price (per gram): 8.10
6.0
Hybrid
Alpha-Pinene,Eudesmols,Guaiol,Myrcene,TrimethylPentanediol
Plain Packaging’s milled indica-dominant hybrid strain is greenhouse-grown and has very mild THC potency. This conveniently pre-ground flower is accented by orange hairs, and its myrcene terpene profile (which is also present in hops and thyme) produces an earthy, clove-like aroma. It’s available in various dried flower sizes.
https://www.ocs.ca/products/cold-creek-kush
Price (per gram): 11.66
19.0
Indica dominant
Beta-Pinene,Nerolidol,Beta-Caryophyllene,Trans-Caryophyllene
Vertical hand-trims and hang-dries its Cold Creek Kush. The indica-dominant indoor-grown strain’s terpene profile (beta-pinene, nerolidol, beta-caryophyllene) gives it an herbal, earthy, piney, woodsy taste. It has strong THC potency and is available in 1 g, 3.5 g and 7 g dried flower sizes.

https://www.ocs.ca/products/bc-delahaze
Price (per gram): 15.65
19.5
Sativa dominant
Alpha-Pinene,Myrcene,Terpinene,Trans-Caryophyllen

Price (per gram): 11.52
14.0
Indica dominant
Alpha-Pinene,Beta-Caryophyllene,Beta-Myrcene,Beta-Pinene,Limonene
Haven St. grows and harvests its indica-dominant No. 402 Blueberry Kush indoors as an irradiated whole flower, then hand-trims and hang-dries the green and purple buds. Its myrcene-forward terpene profile gives it earthy floral notes and a sweet berry aroma. This strain has a medium to high THC potency and comes in a variety of dried flower and pre-rolled formats.
https://www.ocs.ca/products/lola-montes-edison
Price (per gram): 9.96
17.5
Indica dominant
Beta-Caryophyllene,Guaiol,Myrcene,Bisabolol,Humulene
Lola Montes, from Edison, is an indica-dominant strain with very strong THC potency. This strain produces flowers with a tight, dense structure and a two-tone emerald hue, and its terpene profile includes beta-caryophyllene, guaiol, myrcene, bisabolol and humulene. All Organigram products are individually packaged with Integra Boost technology, which preserves freshness and c

Price (per gram): 9.75
19.5
Indica dominant
Alpha-Pinene,Myrcene,Trans-Caryophyllene
Greenhouse-grown, Blueberry by Canaca is an indica strain with very strong THC potency. This green, orange and purple bud has alpha-pinene, myrcene and trans-caryophyllene terpenes, delivering sweet and fruity flavour notes. Blueberry is available in 3.5 g, 7 g and 15 g dried flower varieties and a three-pack of 0.5 g pre-rolls.
https://www.ocs.ca/products/rise-2
Price (per gram): 10.56
15.0
Sativa dominant
Limonene,Linalool
Rise, by Toronto-native brand Tokyo Smoke, is a sativa-dominant hybrid which offers medium THC potency with little to no CBD content. The strain features a fresh herbaceous bouquet with sharp notes of pine derived from its main terpenes, Limonene and Linalool.
https://www.ocs.ca/products/cbd-shark
Price (per gram): 22.66
7.5
Indica dominant
Myrcene
CBD Shark from Whistler Cannabis Co. is an indica-dominant strain with mild THC potency potential and medium CBD levels that’s grown in

,Strain,Url,Description,Price,Strain_type,Terpenes
0,Balance,https://www.ocs.ca/products/balance-solei,Balance from Solei is available pre-rolled and...,9.06,Indica dominant,"Myrcene,Beta-Caryophyllene,Limonene,Beta-Pinen..."
1,God Bud,https://www.ocs.ca/products/god-bud-redecan,God Bud from Redecan has resin-coated buds wit...,9.09,Hybrid,"Beta-Caryophyllene,Trans-Caryophyllene,Humulen..."
2,Alien Dawg,https://www.ocs.ca/products/alien-dawg,An indica-dominant hybrid strain from Northern...,9.25,Indica dominant,"Humulene,Limonene,Trans-Caryophyllene"
3,Dreamweaver,https://www.ocs.ca/products/dream-weaver,"Grown indoors in Paris, Ont., Dreamweaver is a...",9.25,Indica dominant,"Nerolidol,Humulene,Limonene,Myrcene"
4,No. 301 Napali CBD,https://www.ocs.ca/products/napali-cbd,Haven St. grows and harvests No. 301 Napali CB...,11.20,Hybrid,"Alpha-Pinene,Beta-Caryophyllene,Beta-Pinene,Li..."
5,Purple Chitral,https://www.ocs.ca/products/purple-chitral-woo...,Woodstock Purple Chitral has medium THC potenc...,12.65,Indica dominant,"Myrcene,Beta-Pinene,Alpha-Pinene,Guaiol"
6,Banana Split,https://www.ocs.ca/products/banana-split,Banana Split is an indica-dominant hybrid gree...,11.42,Hybrid,"Beta-Pinene,Limonene,Linalool,Terpinolene"
7,Red,https://www.ocs.ca/products/red,"Grown indoors and long-cured in Napanee, Ont.,...",13.15,Hybrid,"Myrcene,Terpineol,Beta-Caryophyllene"
8,Sativa,https://www.ocs.ca/products/sativa,This greenhouse-grown sativa-dominant hybrid s...,8.10,Sativa dominant,"Alpha-Pinene,Alpha-Santalene,Beta-Caryophyllen..."
9,Houndstooth,https://www.ocs.ca/products/houndstooth,Greenhouse-grown Houndstooth from Tweed is a s...,10.05,Sativa dominant,"Alpha-Pinene,Alpha-Santalene,Beta-Caryophyllen..."


In [12]:
OCS_df.to_csv("OCS.csv")

In [13]:
OCS_df.shape

(71, 6)

### Clean up OCS data to get more common strain names

In [14]:
OCS_df

,Strain,Url,Description,Price,Strain_type,Terpenes
0,Balance,https://www.ocs.ca/products/balance-solei,Balance from Solei is available pre-rolled and...,9.06,Indica dominant,"Myrcene,Beta-Caryophyllene,Limonene,Beta-Pinen..."
1,God Bud,https://www.ocs.ca/products/god-bud-redecan,God Bud from Redecan has resin-coated buds wit...,9.09,Hybrid,"Beta-Caryophyllene,Trans-Caryophyllene,Humulen..."
2,Alien Dawg,https://www.ocs.ca/products/alien-dawg,An indica-dominant hybrid strain from Northern...,9.25,Indica dominant,"Humulene,Limonene,Trans-Caryophyllene"
3,Dreamweaver,https://www.ocs.ca/products/dream-weaver,"Grown indoors in Paris, Ont., Dreamweaver is a...",9.25,Indica dominant,"Nerolidol,Humulene,Limonene,Myrcene"
4,No. 301 Napali CBD,https://www.ocs.ca/products/napali-cbd,Haven St. grows and harvests No. 301 Napali CB...,11.20,Hybrid,"Alpha-Pinene,Beta-Caryophyllene,Beta-Pinene,Li..."
5,Purple Chitral,https://www.ocs.ca/products/purple-chitral-woo...,Woodstock Purple Chitral has medium THC potenc...,12.65,Indica dominant,"Myrcene,Beta-Pinene,Alpha-Pinene,Guaiol"
6,Banana Split,https://www.ocs.ca/products/banana-split,Banana Split is an indica-dominant hybrid gree...,11.42,Hybrid,"Beta-Pinene,Limonene,Linalool,Terpinolene"
7,Red,https://www.ocs.ca/products/red,"Grown indoors and long-cured in Napanee, Ont.,...",13.15,Hybrid,"Myrcene,Terpineol,Beta-Caryophyllene"
8,Sativa,https://www.ocs.ca/products/sativa,This greenhouse-grown sativa-dominant hybrid s...,8.10,Sativa dominant,"Alpha-Pinene,Alpha-Santalene,Beta-Caryophyllen..."
9,Houndstooth,https://www.ocs.ca/products/houndstooth,Greenhouse-grown Houndstooth from Tweed is a s...,10.05,Sativa dominant,"Alpha-Pinene,Alpha-Santalene,Beta-Caryophyllen..."
